<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/PyViz_logo_wm_line.png" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 04. Working with tabular data</h2></div>

As we have already discovered, HoloViews elements are simple wrappers around data that provide a semantically meaningful representation. The real power of HoloViews becomes most evident when working with larger, multi-dimensional datasets, whether they are tabular like in a database or CSV file, or gridded like large datasets of images.

Tabular data (also called columnar data) is one of the most common, general, and versatile data formats, corresponding to how data is laid out in a spreadsheet. There are many different ways to put data into a tabular format, but for interactive analysis having [**tidy data**](http://www.jeannicholashould.com/tidy-data-in-python.html) provides flexibility and simplicity. Here we will show how to make your data tidy as a first step, but see [hvPlot](http://hvplot.pyviz.org) for convenient ways to work with non-tidy data directly.

In this tutorial all the information you have learned in the previous sections will finally really pay off. We will discover how to facet data and use different element types to explore and visualize the data contained in a real dataset, using many of the same libraries introduced earlier along with some statistics methods from SciPy:

<div style="margin: 10px">
<a href="http://holoviews.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/holoviews.png"/></a>
<a href="http://bokeh.pydata.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/bokeh.png"/></a>
<a href="http://pandas.pydata.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:140px" src="../assets/pandas.png"/></a>
<a href="http://numpy.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/numpy.png"/></a>
<a href="https://docs.scipy.org/doc/scipy/reference"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/scipy.png"/></a>
</div>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts, dim, Palette
hv.extension('bokeh')

opts.defaults(
    opts.Bars(xrotation=45, tools=['hover']),
    opts.BoxWhisker(width=800, xrotation=30, box_fill_color=Palette('Category20')),
    opts.Curve(width=600, tools=['hover']),
    opts.GridSpace(shared_yaxis=True),
    opts.Scatter(width=800, height=400, color=Palette('Category20'), size=dim('growth')+5, tools=['hover']),
    opts.NdOverlay(legend_position='left'))

## What is tabular, tidy data?

In [ ]:
macro_df = pd.read_csv('../data/macro.csv')
macro_df.head()

For tidy data, the **columns** of the table represent **variables** or **dimensions** and the **rows** represent **observations**. 

The opposite of tidy data is so-called **wide** data. To see what wide data looks like, we can use the pandas ``pivot_table`` method:

In [ ]:
wide = macro_df.pivot_table('unem', 'year', 'country')
wide.head(5)

In this wide format we can see that each column represents the unemployment figures for one country indexed, and each row a particular year. A wide table can represent data very concisely in some cases, but it is difficult to work with in practice because it does not make dimensions easily accessible for plotting or analysis. To go from wide to tidy data you can use the ``pd.melt`` function: 

In [ ]:
melted = pd.melt(wide.reset_index(), id_vars='year',  value_name='unemployment')
melted.head()

## Declaring Dimensions on a Dataset

A HoloViews `Dataset` is similar to a HoloViews Element, but without having the full set of metadata that would let it visualize itself. A Dataset is useful for declaring the set of `Dimension`s along which your data varies, which will later be inherited by any more specific visualizable elements that you create from the Dataset.

A HoloViews `Dimension` is like a mathematical variable, i.e., a quantity that can have different values for different data points.  There are two types of dimensions: **key dimensions** (like mathematical independent variables), and **value dimensions** (like dependent variables). In `macro_df`, the ``country`` and ``year`` are the independent variables (values where measurements were taken), so when we create a Dataset we will declare those as key dimensions.  The remaining columns will automatically be inferred to be value dimensions (quantities measured for each combination of key dimension values):

In [ ]:
macro = hv.Dataset(macro_df, ['country', 'year'])
macro

One of the first things we'll want to do with our Dimensions is to give them more sensible labels using ``redim.label``:

In [ ]:
macro = macro.redim.label(growth='GDP Growth', unem='Unemployment', year='Year', country='Country')

Notice how HoloViews differs from using a plotting library directly in this case -- here we can annotate the data *once* to capture our knowledge about what those columns represent, and those annotations will then feed directly into any plots later derived from this data.  In a plotting program, you would normally need to supply such metadata every single time you make a new plot, which is tedious, error prone, and discourages easy exploration.  In the rest of this tutorial we'll see how we can explore and reveal this annotated data.


## Groupby

The great thing about a tidy tabular Dataset is that we can easily group the data by a particular variable, allowing us to plot or analyze each subset separately. Let's say for instance that we want to break the macro-economic data down by 'year'. Using the groupby method we can easily split the Dataset into subsets by year:

In [ ]:
print(macro.groupby('Year'))

The resulting object has a top-level data structure called a [``HoloMap``](http://holoviews.org/reference/containers/bokeh/HoloMap.html), indexed by year. A HoloMap (like its dynamically generated equivalent [``DynamicMap``](http://holoviews.org/reference/containers/bokeh/DynamicMap.html)) is a potentially many-dimensional indexable container for Elements and Datasets, allowing them to be explored easily.  

However, we cannot visualize this particular HoloMap, because a Dataset has no visual representation.  We haven't yet told HoloViews whether the various dependent variables here are continuous, discrete, binned, or any of the other properties the data could have that would allow a specific Element to be chosen.

## Mapping dimensions to elements

Luckily, as soon as you choose what sort of Element you want a given column to be, you can make this data visualizable using the convenient ``.to`` method, which allows us to group the dataset and map dimensions to elements in a single step.

The ``.to`` method of a Dataset takes up to four main arguments:

1. The element you want to convert to
2. The key dimensions (i.e., independent variables) to display
3. The value dimensions (i.e. dependent variables) to display, if any
4. The dimensions to group by, if any

As a first simple example let's go through such a declaration:

1. We will use a ``Curve`` to declare that the variables are continuous
2. Our independent variable will be the 'year'
3. Our dependent variable will be 'unem'
4. We will ``groupby`` the 'country'.

In [ ]:
curves = macro.to(hv.Curve, 'year', 'unem', groupby='country')
print(curves)
curves

If you look at the printed output you will see that instead of a simple ``Curve`` we got a ``HoloMap`` of ``Curve`` Elements for each country. Each Curve is now visualizable, but we haven't told HoloViews what to do with the ``Country``, and so to make the entire structure visualizable, HoloViews creates a widget where you can select which country you want to view. Additional value dimensions would result in additional widgets here.

Alternatively we could also group by the year and view the unemployment rate by country as Bars instead. If we simply want to groupby all remaining key dimensions (in this case just the year) we can leave out the groupby argument:

In [ ]:
bars = macro.sort('country').to(hv.Bars, 'country', 'unem')
bars.opts(width=600)

In [ ]:

# Exercise: Create a hv.HeatMap using ``macro.to``, declaring kdims 'year' and 'country', and vdims 'growth'
# You'll need to declare ``width`` and/or ``xrotation`` plot options for HeatMap to make the plot readable
# You can also add ``tools=['hover']`` to get more info on each cell


## Displaying distributions

Often we want to summarize the distribution of values, e.g. to reveal the distribution of unemployment rates for each OECD country across all measurements. This means we want to ignore the 'year' dimension in our dataset, letting it be summarized instead. To stop HoloViews from grouping by the extra variable, we pass an empty list to the groupby argument. In this case we can easily declare the ``BoxWhisker`` directly, but omitting a key dimension from the ``groupby`` can be useful in cases when there are more dimensions:

In [ ]:
macro.to(hv.BoxWhisker, 'country', 'growth', groupby=[])
# Is equivalent to:
hv.BoxWhisker(macro, kdims=['country'], vdims=['growth'])

In [ ]:

# Exercise: Display the distribution of GDP growth by year using the BoxWhisker element


## Faceting dimensions

Once the data has been grouped into a ``HoloMap`` as we did above, we can further use the grouping capabilities by using the ``.grid``, ``.layout`` and ``.overlay`` methods to lay the groups out on the page rather than flipping through them with a set of widgets.

#### NdOverlay

In [ ]:
ndoverlay = macro.to(hv.Scatter, 'year', ['unem', 'growth']).overlay()
print(ndoverlay)
ndoverlay.relabel('OECD Unemployment 1960 - 1990')

#### GridSpace

In [ ]:
subset = macro.select(country=['Austria', 'Belgium', 'Netherlands', 'West Germany'])
grid = subset.to(hv.Bars, 'year', 'unem').grid()
print(grid)
grid

To understand what is actually going on here, let's rewrite this example in a slightly different way. Instead of using the convenient ``.to`` or ``.groupby`` methods, we can express the same thing by explicitly iterating over the countries we want to look at, selecting the subset of the data for that country using ``.select`` and then passing these plots to the container we want.

In the example above that means we ``select`` by 'country' on the macro ``Dataset``, pass the selection to ``Bars`` elements, and declare the key and value dimension to display. We then pass the dictionary of ``Bars`` elements to the ``GridSpace`` container and declare the kdim of the container as 'Country':

In [ ]:
countries = ['Austria', 'Belgium', 'Netherlands', 'West Germany']
hv.GridSpace({country: hv.Bars(macro.select(country=country), 'year', 'unem') for country in countries},
             kdims=['Country'])

As you can see, ``.to`` is much simpler and less error-prone in practice.

#### NdLayout

In [ ]:
ndlayout = subset.to(hv.Curve, 'year', 'unem').layout()
print(ndlayout)
ndlayout.opts(
    opts.Curve(width=200, height=200))

In [ ]:

## Exercise: Recreate the plot above using hv.NdLayout and using macro.select just as we did for the GridSpace above


## Aggregating

Another common operation is computing aggregates (summary transformations that collapse some dimensions of the data down to scalar values like a mean or a variance). We can compute and visualize these easily using the ``aggregate`` method. The aggregate method lets you declare the dimension(s) to aggregate by and a function to aggregate with (with an optional secondary function to compute the spread if desired). Once we have computed the aggregate we can simply pass it to the [``Curve``](http://holoviews.org/reference/elements/bokeh/Curve.html) and [``ErrorBars``](http://holoviews.org/reference/elements/bokeh/ErrorBars.html):

In [ ]:
agg = macro.reindex(vdims=['growth']).aggregate('year', function=np.mean, spreadfn=np.std)
hv.Curve(agg) * hv.ErrorBars(agg)

In [ ]:
# Exercise: Display aggregate GDP growth by country, building it up in a series of steps
# Step 1. First, aggregate the data by country rather than by year, using
# np.mean and ss.sem as the function and spreadfn, respectively, then
# make a `Bars` element from the resulting ``agg``

# import scipy.stats as ss



In [ ]:
# Step 2: You should now have a bars plot, but with no error bars. Now add ErrorBars as above. 
# Hint: You'll want to make the plot wider and use an xrotation to see the labels clearly


<!--
agg = macro.reindex(vdims=['growth']).aggregate('year', function=np.mean, spreadfn=np.std)
hv.Curve(agg) * hv.ErrorBars(agg)
-->

## Onward

* Go through the Tabular Data [getting started](http://holoviews.org/getting_started/Tabular_Datasets.html) and [user guide](http://holoviews.org/user_guide/Tabular_Datasets.html).
* Learn about slicing, indexing and sampling in the [Indexing and Selecting Data](http://holoviews.org/user_guide/Indexing_and_Selecting_Data.html) user guide.

The [next section](./05_Working_with_Gridded_Data.ipynb) shows a similar approach, but for working with gridded data, in multidimensional array formats.